In [1]:
import pandas as pd
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.layers import LSTM, Dense

In [2]:
df = pd.read_csv(r'D:\L_IACD\SoundClassification\ML2-Project\UrbanSound8K.csv')
df = df.sort_values("fold")
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
4365,180937-7-3-26.wav,180937,317.584561,321.584561,2,1,7,jackhammer
5709,22962-4-0-2.wav,22962,1.000000,5.000000,1,1,4,drilling
895,118279-8-0-0.wav,118279,0.000000,4.000000,2,1,8,siren
896,118279-8-0-1.wav,118279,0.500000,4.500000,2,1,8,siren
897,118279-8-0-10.wav,118279,5.000000,9.000000,2,1,8,siren
